# Feature selection and feature engineering

This isn't based on a news article, exactly, it's from a paper. You can see the paper in `/data/`.

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)
%matplotlib inline

# Reading in our data

## Every single person in a crash

We'll start by reading in the list of people who were involved in an accident.

**Call this dataframe `people`.**

In [5]:
pwd

'/Users/dronejournalismlab/Desktop/ALGORITHM/HW/HW6-Cars'

In [8]:
df = pd.read_csv('/Users/dronejournalismlab/Desktop/ALGORITHM/HW/HW6-Cars/data/combined-person-data.csv')
df.head()

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID
0,1.0,NaN,0.0,NaN,N,C,0.0,1952-04-20 00:00:00,NaN,0.0,1.0,NaN,1.0,N,1,PA,NaN,NaN,NaN,NaN,NaN,NaN,48dd00ee-e033-47e7-ad1e-0b734020301b,D,AB4284000S,13.0,F,eb6aadb8-dacb-4744-a1a7-ab812c96f27f
1,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593
2,1.0,NaN,0.0,NaN,N,C,0.0,1960-10-04 00:00:00,NaN,0.0,0.0,NaN,0.0,Y,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,f3b2743f-fbc3-4345-9419-56a0ca29102c,D,AB4313000X,13.0,F,3c8629d0-d524-47c1-bfbc-b18e07f3087e
3,1.0,NaN,0.0,NaN,N,B,0.0,1971-05-28 00:00:00,NaN,0.0,1.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,5bbe589b-a2db-4dbb-be9b-17a800d69a08,D,AB4313000Y,0.0,F,c4628cdb-f295-4a24-8a4b-653741ac6ae7
4,1.0,NaN,0.0,NaN,N,D,0.0,1955-04-23 00:00:00,NaN,0.0,0.0,NaN,1.0,N,1,SC,NaN,NaN,NaN,NaN,NaN,NaN,b914136f-5ecd-46bb-94ec-ff5d4136c3eb,D,AB4669001F,13.0,F,cdda1580-fd79-4358-8819-c2250f494591


In [27]:
df.shape

(888283, 28)

How often did each severity of injury show up? (e.g. not injured, non-incapacitating injury, etc)

In [16]:
df.INJ_SEVER_CODE.value_counts()

1    716806
2     82642
3     76801
4     10353
5      1681
Name: INJ_SEVER_CODE, dtype: int64

In [19]:
df.dtypes

AIRBAG_DEPLOYED          float64
ALCOHOL_TESTTYPE_CODE    float64
ALCOHOL_TEST_CODE        float64
BAC_CODE                 float64
CDL_FLAG                  object
CLASS                     object
CONDITION_CODE           float64
DATE_OF_BIRTH             object
DRUG_TESTRESULT_CODE      object
DRUG_TEST_CODE           float64
EJECT_CODE               float64
EMS_UNIT_LABEL            object
EQUIP_PROB_CODE          float64
FAULT_FLAG                object
INJ_SEVER_CODE             int64
LICENSE_STATE_CODE        object
MOVEMENT_CODE            float64
OCC_SEAT_POS_CODE        float64
PED_LOCATION_CODE        float64
PED_OBEY_CODE            float64
PED_TYPE_CODE            float64
PED_VISIBLE_CODE         float64
PERSON_ID                 object
PERSON_TYPE               object
REPORT_NO                 object
SAF_EQUIP_CODE           float64
SEX_CODE                  object
VEHICLE_ID                object
dtype: object

In [21]:
df.INJ_SEVER_CODE.astype(object)

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
888253    2
888254    2
888255    2
888256    2
888257    2
888258    2
888259    2
888260    2
888261    2
888262    2
888263    2
888264    2
888265    2
888266    2
888267    2
888268    2
888269    2
888270    2
888271    2
888272    2
888273    3
888274    3
888275    3
888276    3
888277    3
888278    3
888279    3
888280    3
888281    3
888282    4
Name: INJ_SEVER_CODE, Length: 888283, dtype: object

We're only interested in fatalities, so let's create a new `had_fatality` column for when people received a fatal injury.

**Confirm there were 1681 people with fatal injuries.**

In [25]:


df_fatality = df[df['INJ_SEVER_CODE']== 5]
df_fatality

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID
950,4.0,NaN,0.0,NaN,N,C,0.0,1993-07-17 00:00:00,NaN,0.0,1.0,A,0.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,2f917b9b-6d42-4997-a972-db75d91ec61e,D,CB4718000W,1.0,F,bdb144e3-b96c-4e6d-bc93-34386bb0f7e7
951,4.0,NaN,0.0,NaN,N,C,0.0,1993-01-11 00:00:00,NaN,0.0,2.0,NaN,99.0,N,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,0d9cf44d-1a45-48a9-a3c6-73add5bdaa9a,D,MSP6927001T,99.0,F,2a577dba-0a32-4e2e-a1c3-daeb1d293663
17142,2.0,NaN,0.0,NaN,N,C,1.0,1932-10-03 00:00:00,NaN,0.0,4.0,B,1.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,25523c5b-eaa8-4697-9387-2809c66cb02e,D,BJ0146004Q,13.0,F,c4a686eb-fcf1-4179-8c5c-fbfa94a8841d
17143,2.0,NaN,0.0,NaN,N,C,1.0,1999-12-07 00:00:00,NaN,0.0,4.0,A,1.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,cd6ebd9c-1fdb-4c91-9dbd-029e16ce8316,D,MSP1950007W,13.0,F,9d0684ac-5cf5-465a-93e0-7ec72115a102
17144,4.0,NaN,0.0,NaN,N,C,1.0,1955-09-20 00:00:00,NaN,0.0,4.0,NaN,1.0,U,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,f8fa330f-0fa5-4030-943c-ff259e667dd3,D,MSP25990020,13.0,F,1ad0dbe5-6250-4f26-b801-283989ca936a
17145,1.0,NaN,0.0,NaN,N,C,1.0,1928-08-21 00:00:00,NaN,0.0,4.0,NaN,1.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,eb3841bc-95c0-4c46-86e1-87cc7f532bfc,D,MSP615700DK,13.0,F,c02a9a4e-91af-4d59-8dd7-aad146f08155
17146,3.0,NaN,0.0,NaN,N,C,1.0,1991-10-27 00:00:00,NaN,0.0,1.0,A,1.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,b7028bfd-809e-4b13-bd06-d6bd729eee12,D,MSP617400L7,13.0,F,558a21a1-5807-4104-b928-7d933c251028
17147,2.0,NaN,0.0,NaN,N,C,1.0,1965-05-17 00:00:00,NaN,0.0,1.0,A,1.0,N,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,4bc6153a-229b-415e-ba18-666e3b302a6a,D,MSP6503006M,13.0,F,fee64404-8b8f-4a10-9734-7a377fb1fc33
17148,4.0,NaN,0.0,NaN,N,C,1.0,1981-06-23 00:00:00,NaN,0.0,4.0,NaN,1.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,2bbe45c0-bf12-455e-9f93-f828235dc12c,D,MSP69170002,13.0,F,04175c1b-8181-405b-9af0-4ebcfc7f5d8f
17466,4.0,NaN,0.0,NaN,N,C,2.0,1987-10-29 00:00:00,NaN,0.0,1.0,A,0.0,Y,5,MD,NaN,NaN,NaN,NaN,NaN,NaN,565c123c-7ac4-4dd3-8829-26a1fc439e77,D,CB4718000V,0.0,F,1ac3d902-3057-4137-9afa-ecff7b18018d


In [26]:
df_fatality.shape

(1681, 28)

## Working on Features

### Starting our analysis

We're going to run a regression on the impact of being **male vs female on crash fatalities**. Prepare a dataframe called `train_df` with the appropriate information in it.

* **Tip:** What column(s) are your input, and what is your output? Aka independent and dependent variables
* **Tip:** You'll need to convert your input column into something numeric, I suggest using `.replace`
* **Tip:** We aren't interested in the "Unknown" sex - either filtering or `np.nan` + `.dropna()` might be useful ways to get rid of those columns

In [29]:
train_df = df[['SEX_CODE', 'INJ_SEVER_CODE']]
train_df.head(10)

,SEX_CODE,INJ_SEVER_CODE
0,F,1
1,F,1
2,F,1
3,F,1
4,F,1
5,F,1
6,F,1
7,F,1
8,F,1
9,F,1


In [31]:
train_df['SEX_CODE'] = train_df['SEX_CODE'].replace('F', '1')
train_df.head(5)

/Users/dronejournalismlab/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [34]:
train_df['SEX_CODE'] = train_df['SEX_CODE'].replace('M', '2')
train_df

/Users/dronejournalismlab/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [36]:
train_df.dropna()

,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


Confirm that your `train_df` has two columns and 815,827 rows.

> **Tip:** If you have more rows, make sure you dropped all of the rows with Unknown sex.
>
> **Tip:** If you have more columns, make sure you only have your input and output columns.

In [42]:
#train_df.drop(train_df['SEX_CODE']=="U", axis=1)
train_df = train_df[train_df.SEX_CODE != 'U']
train_df.head(5)

,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [48]:
train_df['SEX_CODE'] = train_df['SEX_CODE'].replace(np.nan, 0)
train_df

,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [51]:
train_df = train_df[train_df.SEX_CODE != 0]
train_df

,SEX_CODE,INJ_SEVER_CODE
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [52]:
train_df.SEX_CODE.value_counts()

2    460973
1    354854
Name: SEX_CODE, dtype: int64

In [53]:
train_df.shape

(815827, 2)

### Run your regression

See the effect of sex on whether the person's injuries are fatal or not. After we train the regression, we can use my ✨favorite technique✨ to display features and their odds ratios:
    
```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)
```

### Use words to interpret this result

## Adding more features

The actual crash data has more details - whether it was snowy/icy, whether it was a highway, etc. 

Read in `combined-crash-data.csv`, calling it **`crashes`**, and merge it with our people dataset. I'll save you a lookup: the `REPORT_NO` is what matches between the two.

### Examining more possible features

How often was it wet, dry, snowy, icy, etc? **What was the most common surfce condition?**

* **Tip:** We're interested in surface condition, _not_ road condition, _not_ weather condition

Do you feel that a **Dry** road condition should be the average of **Wet** and **Snow?**

The answer to that should be *no*, which means we can't use this data as numeric data. We want a different coefficient for each of these - I want to know the impact of dry, the impact of wet, the impact of snow, all separately.

Start by **replacing each code with a proper description.** I'll even include them here:

* `00` - Not Applicable
* `01`	- Wet
* `02`	- Dry
* `03`	- Snow
* `04`	- Ice
* `05`	- Mud, Dirt, Gravel
* `06`	- Slush
* `07`	- Water (standing/moving)
* `08`	- Sand
* `09`	- Oil
* `88`	- Other
* `99`	- Unknown

But watch out, pandas read the column in as numbers so they might have come through a little differently than their codes.

Confirm you have 147,803 wet, and a few codes you can't understand, like `6.03` and `7.01`.

Replace the codes you don't understand with `Other`.

Confirm you have 3,196 'Other'.

## One-hot encoding

We're going to use `pd.get_dummies` to build a variable you'll call `surf_dummies`. Each surface condition should be a `0` or `1` as to whether it was that condition (dry, icy, wet, etc).

Use a `prefix=` so we know they are **surface** conditions.

You'll want to drop the column you'll use as the reference category.

**Before we do this: which column works best as the reference?**

Now build your `surf_dummies` variable.

Confirm your `surf_dummies` looks roughly like this:

|surface_Ice|Surce_Mud, Dirt, Gravel|surface_Not Applicable|...|surface_Wet|
|---|---|---|---|---|
|0|0|0|...|0|
|0|0|0|...|0|
|0|0|1|...|0|
|0|0|1|...|0|
|0|0|0|...|1|

## Another regression

Let's run another regression to see the impact of both **sex and surface condition** on fatalities.

### Build your `train_df`

To build your `train_df`, I recommend doing it either of these two ways. They both first select the important columns, then add in the one-hot encoded `surf_dummies` columns.

```python
train_df = pd.DataFrame({
    'col1': merged.col1,
    'col2': merged.col2,
    'col3': merged.col3,
})
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

or like this:

```python
train_df = train_df[['col1','col2','col3']].copy()
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

The second one is shorter, but the first one makes it easier to use comments to remove columns later.


### Run your regression and check your odds ratios

Actually no, wait, first - what kind of surface do you think will have the **highest fatality rate?**

Confirm your `train_df` has 815,843 rows and 9 columns.

* **Tip:** When you run your regression, if you get an error about not knowing what to do with `U`, it's because you didn't convert your sex to numbers (or if you did, you didn't do it in your original dataframe)

**Is this what you expected?** Why do you think this result might be the case?

# More features: Vehicles

Maybe the car they're in is related to the car they were in. Luckily, we have this information - **read in `combined_vehicle_data` as `vehicles`.**

## Weights of those cars

The car weights are stored in **another file** since the info had to come from an API. I looked up the VINs - vehicle identification numbers - in a government database to try to get data for each of them.

**Read them and build a new dataframe that is both the vehicle data along with their weights.** You can call it `vehicles` since you don't need the original weightless vehicle data any more.

Confirm that your combined `vehicles` dataset should have 534,436 rows and 35 columns. And yes, that's less than we were working with before - you haven't combined it with the people/crashes dataset yet.

### Filter your data

We only want vehicles that are "normal" - somewhere between 1500 and 6000 pounds. Filter your vehicles to only include those in that weight range.

Confirm that you have 532,370 vehicles in the dataset.

## Add this vehicle information to your merged data

Now we'll have a dataframe that contains information on:

* The people themselves and their injuries
* The crash
* The vehicles

Every person came with a `VEHICLE_ID` column that is the vehicle they were in. You'll want to merge on that.

Confirm you have 99 columns and 616,212 rows. **That is a lot of possible features!**

## Another regression, because we can't get enough

Build another `train_df` and run another regression about **how car weight impacts the chance of fatalities**. You'll want to confirm that your dataset has 616,212 and 2 columns.

**Can you translate that into plain English?** Remember weight is in **pounds**.

I feel like pounds isn't the best measure for something like this. Remember how we had to adjust percentages with AP and life expecntancy, and then change around the way we said things? It sounded like this:

> Every 10% increase in unemployment translates to a year and a half loss of life expectancy

Instead of every single pound, maybe we could do every... some other number of pounds? One hundred? One thousand?

**Run another regression with weight in thousands of pounds.** Get another odds ratio. Give me another sentence English.

In [ ]:
# Every thousand pounds heavier your car is increase translates to a 15% decrease in fatalities

# Two-car accidents, struck and striker

Here's the thing, though: **it isn't just the weight of your car.** It's the weight of both cars! If I'm in a big car and I have a wreck with a smaller car, it's the smaller car that's in trouble.

To get that value, we need to do some **feature engineering**, some calculating of *new* variables from our *existing* variables.

We need to jump through some hoops to do that.

##  Two-car accidents

First we're going to count how many vehicles were in each accident. Since we're looking to compare the weight of two cars hitting each other, **we're only going to want crashes with only two cars.**

In [ ]:
counted = vehicles.REPORT_NO.value_counts()
counted.head(10)

By using `.value_counts` I can see how many cars were in each crash, and now I'm going to filter to get a list of all of the ones with two vehicles.

In [ ]:
two_car_report_nos = counted[counted == 2].index
two_car_report_nos

And now we'll filter my vehicles so we only have those that were in two-vehicle crashes.

In [ ]:
vehicles = vehicles[vehicles.REPORT_NO.isin(two_car_report_nos)]

### Struck and striker

To do the math correctly, we need both the risk of someone dying in the smaller car _and_ the risk of someone dying in the bigger car. To do this we need to separate our cars into two groups:

* The 'struck' vehicle: did the person die inside?
* The 'striker' vehicle: how much heavier was it than the struck car?

But we don't know which car was which, so we have to try out both versions - pretending car A was the striker, then pretending car B was the striker. It's hard to explain, but you can read `Pounds That Kill - The External Costs of Vehicle Weight.pdf` for more details on how it works.

In [ ]:
cars_1 = vehicles.drop_duplicates(subset='REPORT_NO', keep='first')
cars_2 = vehicles.drop_duplicates(subset='REPORT_NO', keep='last')

In [ ]:
cars_merged_1 = cars_1.merge(cars_2, on='REPORT_NO', suffixes=['_striker', '_struck'])
cars_merged_2 = cars_2.merge(cars_1, on='REPORT_NO', suffixes=['_striker', '_struck'])
vehicles_complete = pd.concat([cars_merged_1, cars_merged_2])
vehicles_complete.head()

## Put people in their cars

Which car was each person in? We'll assign that now.

In [ ]:
merged = people.merge(vehicles_complete, left_on='VEHICLE_ID', right_on='VEHICLE_ID_struck')
merged.head(3)

# Add the crash details

You did this already! I'm going to do it for you. We're merging on `REPORT_NO_x` because there are so many `REPORT_NO` columns duplicated across our files that pandas started giving them weird names.

In [ ]:
merged = merged.merge(crashes, left_on='REPORT_NO_x', right_on='REPORT_NO')
merged.head(3)

## Filter

We already filtered out vehicles by weight, so we don't have to do that again.

# Calculated features

I'm sure you forgot what all the features are, so we'll bring back whether there was a fatality or not

## Feature: Accident was fatal

In [ ]:
merged['had_fatality'] = (merged.INJ_SEVER_CODE == 5).astype(int)
merged.had_fatality.value_counts()

## Feature: Weight difference

**Remove everything missing weights for strikers or struck vehicles.** You might need to `merged.columns` to remind yourself what the column names are.

Confirm your dataset has roughly 335,000 rows.

Create a new feature called `weight_diff` about how much heavier the striking car was compared to the struck car. **Make sure you've done the math correctly!**

### Feature adjustment

Make all of your weight columns in **thousands of pounds** instead of just in pounds. It'll help you interpret your results much better.

# Another regression!!!

**What is the impact of weight difference on fatality rate?** Create your `train_df`, drop missing values, run your regression, analyze your odds ratios.

Please translate your odds ratio into plain English.

## Adding in more features

How about speed limit? That's important, right? We can add the speed limit of the striking vehicle with `SPEED_LIMIT_striker`.

Can you translate the speed limit odds ratio into plain English?

### Feature engineering: Speed limits

Honestly, that's a pretty bad way to go about things. What's more fun is if we **translate speed limits into bins.**

First, we'll use `pd.cut` to assign each speed limit a category.

In [ ]:
speed_bins = [-np.inf, 10, 20, 30, 40, 50, np.inf]
merged['speed_bin'] = pd.cut(merged.SPEED_LIMIT_struck, bins=speed_bins)
merged[['SPEED_LIMIT_striker', 'speed_bin']].head(10)

Then we'll one-hot encode around 20-30mph speed limits.

In [ ]:
speed_dummies = pd.get_dummies(merged.speed_bin, 
                               prefix='speed').drop('speed_(20.0, 30.0]', axis=1)
speed_dummies.head()

## Running a regression

I like this layout for creating `train_df`, it allows us to easily add dummies and do a little replacing/encoding when we're building binary features like for sex.

> If the below gives you an error, it's because `SEX_CODE` is already a number. In that case, just remove `.replace({'M': 1, 'F': 0, 'U': np.nan })`.

In [ ]:
# Start with our normal features
train_df = pd.DataFrame({
    'weight_diff': merged.weight_diff,
    'sex': merged.SEX_CODE,#.replace({'M': 1, 'F': 0, 'U': np.nan }),
    'had_fatality': merged.had_fatality,
})
# Add the one-hot encoded features
train_df = train_df.join(speed_dummies)
train_df = train_df.join(surf_dummies)
# Drop missing values
train_df = train_df.dropna()
train_df.head()

Describe the impact of the different variables in simple language. What has the largest impact?

## Now you pick the features

Up above you have examples of:

* Creating features from numbers (speed limits)
* Creating features from 0/1 (sex)
* Creating features from binning numbers that are one-hot encoded (speed limit bins - `speed_bins`)
* Creating features from categories that are one-hot encoded (surface - `surf_dummies`

What else do you think matters? Try to plug in more features and see if you can get anything interesting.

> * **Hot tip:** The funniest/most interesting thing feature you can add is also the dumbest. Ask me about it in #algorithms if you end up getting down here.